In [ ]:
%pip install vaderSentiment textblob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

file_path = '/content/GenAI in education Dataset - latest cleaned.csv'
data = pd.read_csv(file_path)

data = data.drop(columns=['link'])

vader_analyzer = SentimentIntensityAnalyzer()

def vader_sentiment(text):
    score = vader_analyzer.polarity_scores(text)['compound']
    if score >= 0.05:
        return 'positive'
    elif score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

def textblob_sentiment(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0:
        return 'positive'
    elif polarity < 0:
        return 'negative'
    else:
        return 'neutral'


data['vader_label'] = data['text_clean'].apply(vader_sentiment)
data['textblob_label'] = data['text_clean'].apply(textblob_sentiment)

# Save comparison CSV
data.to_csv('sentiment_labels_comparison.csv', index=False)

In [ ]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score

# Load the CSV with labels
file_path = 'sentiment_labels_comparison.csv'
data = pd.read_csv(file_path)

# Drop rows with missing values in the sentiment label columns
data_cleaned = data.dropna(subset=['vader_label', 'textblob_label'])

# Calculate Cohen's Kappa
kappa = cohen_kappa_score(data_cleaned['vader_label'], data_cleaned['textblob_label'])

print(f"Cohen's Kappa between VADER and TextBlob labels: {kappa:.3f}")

Cohen's Kappa between VADER and TextBlob labels: 0.307


Ensemble Labelling

In [ ]:
def ensemble_vote(row):
    votes = [row['vader_label'], row['textblob_label']]

    # Count votes
    vote_counts = Counter(votes)

    vote_counts = Counter(votes)

    if vote_counts['positive'] == 2:
        return 'positive'
    elif vote_counts['negative'] == 2:
        return 'negative'
    else:
        return 'negative'

In [ ]:
import pandas as pd
from collections import Counter

df = pd.read_csv("/content/GenAI in education Dataset - sentiment_labels_comparison.csv")

# Apply ensemble voting
df['ensemble_label'] = df.apply(ensemble_vote, axis=1)
df.to_csv("ensemble_sentiment_labels.csv", index=False)